In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from tensorflow import keras
(train_input, train_target), (test_input, test_target)= keras.datasets.fashion_mnist.load_data()

In [4]:
# train 과 valid 분리
from sklearn.model_selection import train_test_split

train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1,28*28)

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size = 0.2, random_state=42
)

---
## Layer를 추가하는 방법 -1

In [5]:
# Layer 만들기
dense1 = keras.layers.Dense(100, activation = 'sigmoid', input_shape=(784,)) # 은닉층, # Dense : 밀집층
dense2 = keras.layers.Dense(10, activation = 'softmax') # 출력층

In [6]:
# 전체 모델에 층 추가
model = keras.Sequential([dense1,dense2])

2022-07-20 14:36:48.542458: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
# Summary
model.summary()
# dense (Dense)               (None, 100)               78500   은닉층
# dense_1 (Dense)             (None, 10)                1010   출력층
# Param = 784 * 100, 100 * 10 + 10(절편이 더해져서)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


---
## Layer를 추가하는 방법 -2

In [8]:
model = keras.Sequential(
    [
        keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name = 'hidden'),
        keras.layers.Dense(10, activation = 'softmax', name ='output')
    ], name = '패션 Mnist 모델'
)

In [9]:
model.summary()

Model: "패션 Mnist 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


---
## Layer를 추가하는 방법 -3

In [11]:
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation ='sigmoid', input_shape =(784,)))
model.add(keras.layers.Dense(10, activation ='softmax'))

In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 100)               78500     
                                                                 
 dense_3 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


---
## 모델 훈련

In [13]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.5627 - accuracy: 0.8085
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.4058 - accuracy: 0.8539
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3720 - accuracy: 0.8649
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3508 - accuracy: 0.8731
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3345 - accuracy: 0.8783


---
- Sigmoid 함수는 숫자가 무한대로 커지게 되면 데이터와 차이가 거의 없다는 단점이 있다.
- 이를 해결할 것이 Relu함수 인데 인공신경망에 기여한 바가 크며 음수는 모든0으로 양수는 항상 최대값을 출력하는 함수.

In [14]:
# 입력층을 단수화 하기
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28))) # 입력층을 2차원 원래대로 사용한다.
model.add(keras.layers.Dense(100, activation = 'relu'))
model.add(keras.layers.Dense(10, activation = 'softmax'))


In [15]:
model.summary()
# (None, 784) 행은 모르겠고, 열은 784개이다.

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               78500     
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


---
## Optimizer(최적화 알고리즘)
- 훈련시 Optimizer를 사용하여 최적화 알고리즘을 구성한다.
- 보통 사용하는것이 확률적 경사하강법 Optimizer 이다.

In [16]:
# 적용 방법 (확률적 경사 하강법) -1
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics='accuracy')

In [17]:
# 적용 방법 (확률적 경사 하강법) -2
sgd = keras.optimizers.SGD()
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')

In [18]:
# 적용 방법 (확률적 경사 하강법) + 학습률 변경 -3 : SGD의 learning_rate의 기본값은 0.01
sgd = keras.optimizers.SGD(learning_rate=0.1)
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')

In [19]:
# 적용 방법 (Momentum)
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')

In [20]:
# 적용 방법 (Adagrad)
adagrad =  keras.optimizers.Adagrad()
model.compile(optimizer=adagrad, loss='sparse_categorical_crossentropy', metrics='accuracy')

In [21]:
# 적용 방법 (RMSprop)
rmsprop = keras.optimizers.RMSprop()
model.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy', metrics='accuracy')